<a href="https://colab.research.google.com/github/Yash0330/Callbacks-in-Keras/blob/master/Introduction_to_callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to callbacks in keras

Hi everyone my name is yashwanth let's see why callbacks is needed and how to implement it.

Building Deep Learning models without callbacks is like driving a car with no functioning brakes — you have little to no control over the whole process that is very likely to result in a disaster. In this article, you will learn how to monitor and improve your Deep Learning models using Keras callbacks like ModelCheckpoint and EarlyStopping.

Often, when training a very deep neural network, we want to stop training once the training accuracy reaches a certain desired threshold. Thus, we can achieve what we want (optimal model weights) and avoid wastage of resources (time and computation power).
In this brief tutorial, let’s learn how to achieve this in Tensorflow and Keras.


## What is callback in keras

A callback is a set of functions to be applied at given stages of the training procedure. You can use callbacks to get a view on internal states and statistics of the model during training.

A custom callback is a powerful tool to customize the behavior of a Keras model during training, evaluation, or inference, including reading/changing the Keras model. Examples include tf.keras.callbacks.TensorBoard where the training progress and results can be exported and visualized with TensorBoard, or tf.keras.callbacks.ModelCheckpoint where the model is automatically saved during training, and more. In this guide, you will learn what Keras callback is, when it will be called, what it can do, and how you can build your own. Towards the end of this guide, there will be demos of creating a couple of simple callback applications to get you started on your custom callback.

In [0]:
# Import tensorflow
import tensorflow as tf

## First I will say how to stop training a neural-network using callback

1.First, set the accuracy threshold till which you want to train your model.

In [0]:
acc_thresh = 0.96

2.For implementing the callback first you have to create class and function.

Now, let us implement it to stop training when accuracy reaches acc_thresh.

In [0]:
class myCallback(tf.keras.callbacks.Callback): 
    def on_epoch_end(self, epoch, logs={}): 
        if(logs.get('acc') > ACCURACY_THRESHOLD):   
          print("\nWe have reached %2.2f%% accuracy, so we will stopping training." %(acc_thresh*100))   
        self.model.stop_training = True

Now you may wonder what is going on here?

We are creating new class by extending tf.keras.callbacks.Callback and implementing the on_epoch_end() method which will invoke at the end of epoch.

Next, we are fetching the value of accuracy at the end of that epoch, and if it is greater than our threshold, we are setting the stop_training of model to True using the “self” keyword to access the attributes and methods of the class in python.

3.Now, let us create the instance of an object of myCallback class.

In [0]:
callbacks = myCallback()

4.Next, build a Neural Network or Conv-Net or any model following the normal steps of TensorFlow or Keras.You can pass a callbacks (as the keyword argument callbacks) to any of tf.keras.Model.fit(), tf.keras.Model.evaluate(), and tf.keras.Model.predict() methods. The methods of the callbacks will then be called at different stages of training/evaluating/inference.

callbacks=[the newly instantiated object of myCallback class] 

In [0]:
model.fit(x_train, y_train, epochs=20, callbacks=[callbacks])

And that’s all! While training, as soon as accuracy reaches the value set in acc_thresh, training will be stopped.


 In the above example you can also use on_epoch_begin() which is called at the beginning of an epoch during training.


## Saving the model automatically in training.
Let's say you want to save your model when validation accuracy reaches minimum in between training before overfitting this can be implemented using the ModelCheckpoint in callback which can be done as shown below. 

ModelCheckpoint
 
This callback saves the model after every epoch. Here are some relevant metrics:

filepath: the file path you want to save your model in


monitor: the value being monitored


save_best_only: set this to True if you do not want to overwrite the latest best model


mode: auto, min, or max. For example, you set mode=’min’ if the monitored value is val_loss and you want to minimize it.

In [0]:
from keras.callbacks import ModelCheckpoint
best_model = ModelCheckpoint(filepath,
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

In [0]:
from keras.callbacks import ModelCheckpoint
# autosave best Model
best_model_file = "vgg.h5"
best_model = ModelCheckpoint(best_model_file, monitor='val_acc', verbose=1, save_best_only=True)

Here we are monitoring the validation accuracy say you want training accuracy then just change the monitor value as 'acc' as shown below you can also put 'loss' for training loss and 'val_loss' for validation accuracy.

```
best_model = ModelCheckpoint(best_model_file, monitor='acc', verbose=1, save_best_only=True)
```



This can be passed same as last example.

In [0]:
model.fit(x_train, y_train, epochs=20, validation_split=0.1,callbacks=[best_model])

## LearningRateScheduler

This one is pretty straightforward: it adjusts the learning rate over time using a schedule that you already write beforehand. This function returns the desired learning rate (output) based on the current epoch (epoch index as input).

In [0]:
from keras.callbacks import LearningRateScheduler
lrs = LearningRateScheduler(schedule, verbose=0) # schedule is a function

## Early stopping at minimum loss and Usage of logs dict

Overfitting is a nightmare for Machine Learning practitioners. One way to avoid overfitting is to terminate the process early. The EarlyStoppingfunction has various metrics/arguments that you can modify to set up when the training process should stop. Here are some relevant metrics:

monitor: value being monitored, i.e: val_loss


min_delta: minimum change in the monitored value. For example, min_delta=1 means that the training process will be stopped if the absolute change of the monitored value is less than 1


patience: number of epochs with no improvement after which training will be stopped


restore_best_weights: set this metric to True if you want to keep the best weights once stopped


The code example below will define an EarlyStopping function that tracks the val_loss value, stops the training if there are no changes towards val_loss after 3 epochs, and keeps the best weights once the training stops:

In [0]:
from keras.callbacks import EarlyStopping
earlystop = EarlyStopping(monitor = 'val_loss',
                          min_delta = 0,
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf


Let's build a small model

In [0]:
# Define the Keras model to add callbacks to
def get_model():
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Dense(1, activation = 'linear', input_dim = 784))
  model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.1), loss='mean_squared_error', metrics=['mae'])
  return model

Then, we will load the MNIST data for training and testing from Keras datasets API:

In [0]:
# Load example MNIST data and pre-process it
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

The logs dict contains the loss value, and all the metrics at the end of a batch or epoch. Example includes the loss and mean absolute error.

In [0]:
class LossAndErrorPrintingCallback(tf.keras.callbacks.Callback):

  def on_train_batch_end(self, batch, logs=None):
    print('For batch {}, loss is {:7.2f}.'.format(batch, logs['loss']))

  def on_test_batch_end(self, batch, logs=None):
    print('For batch {}, loss is {:7.2f}.'.format(batch, logs['loss']))

  def on_epoch_end(self, epoch, logs=None):
    print('The average loss for epoch {} is {:7.2f} and mean absolute error is {:7.2f}.'.format(epoch, logs['loss'], logs['mae']))
    
model = get_model()
_ = model.fit(x_train, y_train,
          batch_size=64,
          steps_per_epoch=5,
          epochs=3,
          verbose=0,
          callbacks=[LossAndErrorPrintingCallback()])

For batch 0, loss is   33.11.
For batch 1, loss is 1044.98.
For batch 2, loss is   28.39.
For batch 3, loss is   11.97.
For batch 4, loss is    7.52.
The average loss for epoch 0 is  225.19 and mean absolute error is    8.96.
For batch 0, loss is    6.47.
For batch 1, loss is    6.41.
For batch 2, loss is    4.25.
For batch 3, loss is    6.59.
For batch 4, loss is    4.22.
The average loss for epoch 1 is    5.59 and mean absolute error is    1.95.
For batch 0, loss is    5.12.
For batch 1, loss is    5.22.
For batch 2, loss is    5.25.
For batch 3, loss is    5.40.
For batch 4, loss is    4.37.
The average loss for epoch 2 is    5.07 and mean absolute error is    1.83.


Belowshowcases the creation of a Callback that stops the Keras training when the minimum of loss has been reached by mutating the attribute model.stop_training (boolean). Optionally, the user can provide an argument patience to specify how many epochs the training should wait before it eventually stops.

Now let us create class for it and pass it to callbacks.

In [0]:
import numpy as np

class EarlyStoppingAtMinLoss(tf.keras.callbacks.Callback):
  """Stop training when the loss is at its min, i.e. the loss stops decreasing.

  Arguments:
      patience: Number of epochs to wait after min has been hit. After this
      number of no improvement, training stops.
  """

  def __init__(self, patience=0):
    super(EarlyStoppingAtMinLoss, self).__init__()

    self.patience = patience

    # best_weights to store the weights at which the minimum loss occurs.
    self.best_weights = None

  def on_train_begin(self, logs=None):
    # The number of epoch it has waited when loss is no longer minimum.
    self.wait = 0
    # The epoch the training stops at.
    self.stopped_epoch = 0
    # Initialize the best as infinity.
    self.best = np.Inf

  def on_epoch_end(self, epoch, logs=None):
    current = logs.get('loss')
    if np.less(current, self.best):
      self.best = current
      self.wait = 0
      # Record the best weights if current results is better (less).
      self.best_weights = self.model.get_weights()
    else:
      self.wait += 1
      if self.wait >= self.patience:
        self.stopped_epoch = epoch
        self.model.stop_training = True
        print('\n Restoring model weights from the end of the best epoch.')
        self.model.set_weights(self.best_weights)

  def on_train_end(self, logs=None):
    if self.stopped_epoch > 0:
      print('Epoch %05d: early stopping' % (self.stopped_epoch + 1))

In [0]:
model = get_model()
_ = model.fit(x_train, y_train,
          batch_size=64,
          steps_per_epoch=5,
          epochs=30,
          verbose=0,
          callbacks=[LossAndErrorPrintingCallback(),EarlyStoppingAtMinLoss()])

For batch 0, loss is   34.13.
For batch 1, loss is  811.08.
For batch 2, loss is   22.56.
For batch 3, loss is    7.79.
For batch 4, loss is   11.28.
The average loss for epoch 0 is  177.37 and mean absolute error is    8.24.
For batch 0, loss is    6.41.
For batch 1, loss is    5.56.
For batch 2, loss is    5.50.
For batch 3, loss is    4.42.
For batch 4, loss is    4.99.
The average loss for epoch 1 is    5.38 and mean absolute error is    1.89.
For batch 0, loss is    4.15.
For batch 1, loss is    4.79.
For batch 2, loss is    4.93.
For batch 3, loss is    5.29.
For batch 4, loss is    4.05.
The average loss for epoch 2 is    4.64 and mean absolute error is    1.74.
For batch 0, loss is    5.63.
For batch 1, loss is    6.46.
For batch 2, loss is    8.21.
For batch 3, loss is   13.19.
For batch 4, loss is   35.99.
The average loss for epoch 3 is   13.90 and mean absolute error is    2.88.

 Restoring model weights from the end of the best epoch.
Epoch 00004: early stopping


Now, as you can see above as the loss is increased or did.t improve it stopped.

## TensorBoard

This is the best of all callbacks. By using a TensorBoard callback, logs will be written to a directory that you can then examine with TensorFlow's excellent TensorBoard visualization tool.

In [0]:
keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0,  
          write_graph=True, write_images=True)

This line creates a Callback Tensorboard object, you should capture that object and give it to the fit function of your model.

In [0]:
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)
...
model.fit(...inputs and parameters..., callbacks=[tbCallBack])

This way you gave your callback object to the function. It will be run during the training and will output files that can be used with tensorboard.
If you want to visualize the files created during training, run in your terminal

In [0]:
tensorboard --logdir path_to_current_dir/Graph